In [1]:
import MVTecAD_loaddata as mv
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.datasets import mnist
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape
from keras.layers import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
#モデルの可視化
#from tensorflow.python.keras.utils.vis_utils import plot_model

from keras.utils.vis_utils import plot_model
from keras.models import load_model	
import glob

# Kerasライブラリのインポート
import keras
from keras.datasets import cifar10
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, Input, BatchNormalization, LeakyReLU, Conv2DTranspose, Conv2D

import os
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, UpSampling2D, Conv2D, Conv2DTranspose, Reshape, Input
from tensorflow.keras import optimizers
from efficientnet.tfkeras import EfficientNetB2, preprocess_input
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
import MVTecAD_loaddata as mv
import pandas as pd
from random import randint

In [21]:
def train_gan_data(X_train):
    # ハイパーパラメータの設定
    # バッチサイズ
    batch_size = 32
    # エポック数
    epochs = 3
    # 学習率
    lr = 0.001

    # Generatorモデルの構築
    def build_generator():
        # Generatorは、潜在変数からRGB画像を生成するモデルです。
        # Conv2DTranspose層とBatchNormalization層とLeakyReLU層を用いて、アップサンプリングを行います。
        model = Sequential()
        # 入力層
        model.add(Flatten(input_shape=(256, 256, 3)))
#         model.add(Dense(8*8*512, input_dim=(256, 256, 3)))
        model.add(Reshape((16, 16, 768)))
        # 中間層1
        model.add(Conv2DTranspose(256, kernel_size=5, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層2
        model.add(Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層3
        model.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層4
        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 出力層
        model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh'))
        model.summary()
        return model

    # Discriminatorモデルの構築
    def build_discriminator():
        # Discriminatorは、RGB画像が本物か偽物かを判定するモデルです。
        # Conv2D層とBatchNormalization層とLeakyReLU層を用いて、ダウンサンプリングを行います。
        model = Sequential()
        # 入力層
        model.add(Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(256, 256 ,3)))
        model.add(LeakyReLU(alpha=0.2))
        # 中間層1
        model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層2
        model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層3
        model.add(Conv2D(512, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 出力層
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        return model

    # Generatorモデルの作成
    generator = build_generator()
    # Discriminatorモデルの作成
    discriminator = build_discriminator()
    # Discriminatorモデルのコンパイル
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
    # Discriminatorモデルの学習を停止
    discriminator.trainable = False

    # GANモデルの構築
    # GANモデルは、GeneratorモデルとDiscriminatorモデルを結合したモデルです。
    # 潜在変数を入力として、Discriminatorモデルの出力を得るようにします。
    gan_input = Input(shape=(256, 256, 3))
    gan_output = discriminator(generator(gan_input))
    gan = Model(gan_input, gan_output)
    # GANモデルのコンパイル
    gan.compile(loss='binary_crossentropy', optimizer=Adam(lr))

    # 学習のループ
    for epoch in range(epochs):
        # バッチごとに学習
        for i in range(0, len(X_train), batch_size):
            # 本物の画像データをバッチサイズ分取得
            real_images = X_train[i:i+batch_size]
            # 偽物の画像データをバッチサイズ分生成
            latent_vectors = np.random.normal(0, 1, size=(len(real_images), 256, 256, 3))
            fake_images = generator.predict(latent_vectors)
            # 本物の画像データと偽物の画像データを結合
            images = np.concatenate([real_images, fake_images])
            # 本物の画像データにはラベル1を、偽物の画像データにはラベル0を付ける
            labels = np.concatenate([np.ones((len(real_images), 1)), np.zeros((len(real_images), 1))])
            # ラベルにわずかなノイズを加える
#             labels += 0.05 * np.random.random(labels.shape)
            # Discriminatorモデルの学習を再開
            discriminator.trainable = True
            # Discriminatorモデルの学習と評価
            loss, acc = discriminator.train_on_batch(images, labels)
            # Discriminatorモデルの学習を停止
            discriminator.trainable = False
            # 潜在変数をバッチサイズ分生成
            latent_vectors = np.random.normal(0, 1, size=(len(real_images), 256, 256, 3))
            # 偽物の画像データにはラベル1を付ける（騙す）
            misleading_labels = np.ones((len(real_images), 1))
            # GANモデルの学習
            gan.train_on_batch(latent_vectors, misleading_labels)
#             gan.train_on_batch(images, labels)
        # エポックごとに結果を表示
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')
    imageslatent_vectors = np.random.normal(0, 1, size=(int(len(X_train)*0.125), 256, 256, 3))
    fake_images = generator.predict(imageslatent_vectors)
    x_train = np.concatenate([X_train, fake_images])
    y_train = np.ones((len(x_train), 1))
    y_train[len(X_train):] = -1
    return x_train, y_train, generator

def make_mask(X):
    X_dam = X
    for i in range(len(X_dam)):
        rand_x = randint(32,224)
        rand_y = randint(32,224)
        x1, x2 = rand_x-20, rand_x+20
        y1, y2 = rand_y-20, rand_y+20
        X_dam[i,x1:x2, y1:y2, :] = 1.0
    return X_dam

def save_image_bottle(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/bottle/'+str(i)+'.png', fake[i])

def save_image_cable(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/cable/'+str(i)+'.png', fake[i])

def save_image_capsule(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/capsule/'+str(i)+'.png', fake[i])

def save_image_hazelnut(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/hazelnut/'+str(i)+'.png', fake[i])

def save_image_metal_nut(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/metal-nut/'+str(i)+'.png', fake[i])

def save_image_pill(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/pill/'+str(i)+'.png', fake[i])

def save_image_screw(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/screw/'+str(i)+'.png', fake[i])

def save_image_toothbrush(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/toothbrush/'+str(i)+'.png', fake[i])
        
def save_image_transistor(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/transistor/'+str(i)+'.png', fake[i])

def save_image_zipper(fake):
    for i in range(len(fake)):
        cv2.imwrite('fakes/zipper/'+str(i)+'.png', fake[i]) 


In [22]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_dam = make_mask(X_train)
x_train, y_train, G = train_gan_data(X_dam)
# x_train = np.reshape(X_train, (len(X_train), -1))
# x_test = np.reshape(X_test, (len(X_test), -1))
# result = OCSVM(x_train, x_test, Y_train, y_test)
# res.append(result)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 196608)            0         
                                                                 
 reshape_2 (Reshape)         (None, 16, 16, 768)       0         
                                                                 
 conv2d_transpose_10 (Conv2D  (None, 16, 16, 256)      4915456   
 Transpose)                                                      
                                                                 
 batch_normalization_14 (Bat  (None, 16, 16, 256)      1024      
 chNormalization)                                                
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose_11 (Conv2D  (None, 32, 32, 256)     

KeyboardInterrupt: 

In [18]:
img = G.predict(X_train)

7/7 [==============================] - 16s 2s/step


In [19]:
img_255 = img/np.max(img)*255

In [20]:
for i in range(len(img_255)):
    cv2.imwrite('newfakes/bottle/'+str(i)+'.png', img_255[i])

In [5]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 853ms/step
Epoch 0, Loss: -1.3831748962402344, Accuracy: 0.0
1/1 [==============================] - 1s 845ms/step
Epoch 1, Loss: -4.846161842346191, Accuracy: 0.0
1/1 [==============================] - 1s 854ms/step
Epoch 2, Loss: -6.080794334411621, Accuracy: 0.0
1/1 [==============================] - 1s 840ms/step
Epoch 3, Loss: 4.2958526611328125, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 4, Loss: -13.899402618408203, Accuracy: 0.0
1/1 [==============================] - 1s 849ms/step
Epoch 5, Loss: -19.7257080078125, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 6, Loss: -26.74650764465332, Accuracy: 0.0
1/1 [==============================] - 1s 847ms/step
Epoch 7, Loss: -25.91905975341797, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 8, Loss: -34.243064880371094, Accuracy: 0.0
1/1 [==============================] - 1s 843ms/step
Epoch 9, Loss: -44.60622406

1/1 [==============================] - 1s 843ms/step
Epoch 38, Loss: -120.11016082763672, Accuracy: 0.0
1/1 [==============================] - 1s 858ms/step
Epoch 39, Loss: -122.94640350341797, Accuracy: 0.0
1/1 [==============================] - 1s 851ms/step
Epoch 40, Loss: -130.7610626220703, Accuracy: 0.0
1/1 [==============================] - 1s 850ms/step
Epoch 41, Loss: -155.21456909179688, Accuracy: 0.0
1/1 [==============================] - 1s 849ms/step
Epoch 42, Loss: -172.54351806640625, Accuracy: 0.0
1/1 [==============================] - 1s 854ms/step
Epoch 43, Loss: -171.58016967773438, Accuracy: 0.0
1/1 [==============================] - 1s 844ms/step
Epoch 44, Loss: -177.66017150878906, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 45, Loss: -181.1589813232422, Accuracy: 0.0
1/1 [==============================] - 1s 851ms/step
Epoch 46, Loss: -223.62371826171875, Accuracy: 0.0
1/1 [==============================] - 1s 859ms/step
Epoch 47, Los

1/1 [==============================] - 1s 853ms/step
Epoch 77, Loss: -524.837890625, Accuracy: 0.0
1/1 [==============================] - 1s 842ms/step
Epoch 78, Loss: -589.80810546875, Accuracy: 0.0
1/1 [==============================] - 1s 860ms/step
Epoch 79, Loss: -619.7037963867188, Accuracy: 0.0
1/1 [==============================] - 1s 845ms/step
Epoch 80, Loss: -509.45947265625, Accuracy: 0.0
1/1 [==============================] - 1s 842ms/step
Epoch 81, Loss: -550.4091796875, Accuracy: 0.0
1/1 [==============================] - 1s 840ms/step
Epoch 82, Loss: 1991.704345703125, Accuracy: 0.0
1/1 [==============================] - 1s 843ms/step
Epoch 83, Loss: -212.9151611328125, Accuracy: 0.0
1/1 [==============================] - 1s 849ms/step
Epoch 84, Loss: -440.6601867675781, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 85, Loss: -352.1654052734375, Accuracy: 0.0
1/1 [==============================] - 1s 843ms/step
Epoch 86, Loss: -470.14212036132

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.6965937762825904
f1_score :  0.6540880503144655
accuracy_score :  0.6071428571428571
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [6]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 905ms/step
Epoch 0, Loss: -2.2520601749420166, Accuracy: 0.0
1/1 [==============================] - 1s 718ms/step
Epoch 1, Loss: -2.7339460849761963, Accuracy: 0.0
1/1 [==============================] - 1s 715ms/step
Epoch 2, Loss: -5.401944160461426, Accuracy: 0.0
1/1 [==============================] - 1s 717ms/step
Epoch 3, Loss: -7.483572006225586, Accuracy: 0.0
1/1 [==============================] - 1s 719ms/step
Epoch 4, Loss: -12.991945266723633, Accuracy: 0.0
1/1 [==============================] - 1s 720ms/step
Epoch 5, Loss: -17.706588745117188, Accuracy: 0.0
1/1 [==============================] - 1s 717ms/step
Epoch 6, Loss: -15.471385955810547, Accuracy: 0.0
1/1 [==============================] - 1s 710ms/step
Epoch 7, Loss: -22.13002586364746, Accuracy: 0.0
1/1 [==============================] - 1s 750ms/step
Epoch 8, Loss: -27.06702423095703, Accuracy: 0.0
1/1 [==============================] - 1s 715ms/step
Epoch 9, Loss: -28.70579

1/1 [==============================] - 1s 714ms/step
Epoch 38, Loss: -420.31683349609375, Accuracy: 0.0
1/1 [==============================] - 1s 717ms/step
Epoch 39, Loss: -401.821044921875, Accuracy: 0.0
1/1 [==============================] - 1s 714ms/step
Epoch 40, Loss: -454.9928894042969, Accuracy: 0.0
1/1 [==============================] - 1s 713ms/step
Epoch 41, Loss: -512.0335083007812, Accuracy: 0.0
1/1 [==============================] - 1s 707ms/step
Epoch 42, Loss: -471.1085510253906, Accuracy: 0.0
1/1 [==============================] - 1s 729ms/step
Epoch 43, Loss: -469.39605712890625, Accuracy: 0.0
1/1 [==============================] - 1s 710ms/step
Epoch 44, Loss: -409.2522888183594, Accuracy: 0.0
1/1 [==============================] - 1s 721ms/step
Epoch 45, Loss: 7864.509765625, Accuracy: 0.0
1/1 [==============================] - 1s 720ms/step
Epoch 46, Loss: -164.30674743652344, Accuracy: 0.0
1/1 [==============================] - 1s 722ms/step
Epoch 47, Loss: -284.4

1/1 [==============================] - 1s 718ms/step
Epoch 77, Loss: -684.5458374023438, Accuracy: 0.0
1/1 [==============================] - 1s 718ms/step
Epoch 78, Loss: -626.1719970703125, Accuracy: 0.0
1/1 [==============================] - 1s 711ms/step
Epoch 79, Loss: -647.77294921875, Accuracy: 0.0
1/1 [==============================] - 1s 710ms/step
Epoch 80, Loss: -600.8134765625, Accuracy: 0.0
1/1 [==============================] - 1s 719ms/step
Epoch 81, Loss: -614.0336303710938, Accuracy: 0.0
1/1 [==============================] - 1s 708ms/step
Epoch 82, Loss: -702.9174194335938, Accuracy: 0.0
1/1 [==============================] - 1s 709ms/step
Epoch 83, Loss: -688.4510498046875, Accuracy: 0.0
1/1 [==============================] - 1s 718ms/step
Epoch 84, Loss: -676.5613403320312, Accuracy: 0.0
1/1 [==============================] - 1s 710ms/step
Epoch 85, Loss: -838.85400390625, Accuracy: 0.0
1/1 [==============================] - 1s 710ms/step
Epoch 86, Loss: -605.574523

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.5309134423613882
f1_score :  0.2975206611570248
accuracy_score :  0.3560606060606061
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [7]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 833ms/step
Epoch 0, Loss: -1.8642467260360718, Accuracy: 0.0
1/1 [==============================] - 1s 645ms/step
Epoch 1, Loss: -4.561442852020264, Accuracy: 0.0
1/1 [==============================] - 1s 655ms/step
Epoch 2, Loss: 150.029296875, Accuracy: 0.0
1/1 [==============================] - 1s 640ms/step
Epoch 3, Loss: -8.251079559326172, Accuracy: 0.0
1/1 [==============================] - 1s 640ms/step
Epoch 4, Loss: -9.632800102233887, Accuracy: 0.0
1/1 [==============================] - 1s 647ms/step
Epoch 5, Loss: -11.758099555969238, Accuracy: 0.0
1/1 [==============================] - 1s 686ms/step
Epoch 6, Loss: -18.77004051208496, Accuracy: 0.0
1/1 [==============================] - 1s 647ms/step
Epoch 7, Loss: -20.53348159790039, Accuracy: 0.0
1/1 [==============================] - 1s 662ms/step
Epoch 8, Loss: -22.36964225769043, Accuracy: 0.0
1/1 [==============================] - 1s 649ms/step
Epoch 9, Loss: -8.07610511779785

1/1 [==============================] - 1s 651ms/step
Epoch 31, Loss: -23.449501037597656, Accuracy: 0.0
1/1 [==============================] - 1s 642ms/step
Epoch 32, Loss: -17.173362731933594, Accuracy: 0.0
1/1 [==============================] - 1s 636ms/step
Epoch 33, Loss: -26.46455955505371, Accuracy: 0.0
1/1 [==============================] - 1s 643ms/step
Epoch 34, Loss: -22.734533309936523, Accuracy: 0.0
1/1 [==============================] - 1s 643ms/step
Epoch 35, Loss: -33.314823150634766, Accuracy: 0.0
1/1 [==============================] - 1s 642ms/step
Epoch 36, Loss: -49.29389572143555, Accuracy: 0.0
1/1 [==============================] - 1s 640ms/step
Epoch 37, Loss: -32.634822845458984, Accuracy: 0.0
1/1 [==============================] - 1s 655ms/step
Epoch 38, Loss: -39.701717376708984, Accuracy: 0.0
1/1 [==============================] - 1s 638ms/step
Epoch 39, Loss: -46.339019775390625, Accuracy: 0.0
1/1 [==============================] - 1s 639ms/step
Epoch 40, Los

1/1 [==============================] - 1s 644ms/step
Epoch 62, Loss: -66.5453872680664, Accuracy: 0.0
1/1 [==============================] - 1s 642ms/step
Epoch 63, Loss: -70.49396514892578, Accuracy: 0.0
1/1 [==============================] - 1s 677ms/step
Epoch 64, Loss: -82.7355728149414, Accuracy: 0.0
1/1 [==============================] - 1s 645ms/step
Epoch 65, Loss: -81.81072235107422, Accuracy: 0.0
1/1 [==============================] - 1s 638ms/step
Epoch 66, Loss: -89.1669921875, Accuracy: 0.0
1/1 [==============================] - 1s 638ms/step
Epoch 67, Loss: -79.9260482788086, Accuracy: 0.0
1/1 [==============================] - 1s 640ms/step
Epoch 68, Loss: -108.20947265625, Accuracy: 0.0
1/1 [==============================] - 1s 638ms/step
Epoch 69, Loss: -103.71627044677734, Accuracy: 0.0
1/1 [==============================] - 1s 642ms/step
Epoch 70, Loss: -78.53815460205078, Accuracy: 0.0
1/1 [==============================] - 1s 635ms/step
Epoch 71, Loss: -106.1044921

1/1 [==============================] - 1s 643ms/step
Epoch 93, Loss: -204.49964904785156, Accuracy: 0.0
1/1 [==============================] - 1s 645ms/step
Epoch 94, Loss: -263.8601379394531, Accuracy: 0.0
1/1 [==============================] - 1s 642ms/step
Epoch 95, Loss: -210.09510803222656, Accuracy: 0.0
1/1 [==============================] - 1s 696ms/step
Epoch 96, Loss: -317.1344299316406, Accuracy: 0.0
1/1 [==============================] - 1s 639ms/step
Epoch 97, Loss: -295.7660827636719, Accuracy: 0.0
1/1 [==============================] - 1s 642ms/step
Epoch 98, Loss: -285.0166320800781, Accuracy: 0.0
1/1 [==============================] - 1s 638ms/step
Epoch 99, Loss: -336.3257141113281, Accuracy: 0.0
2/2 [==============================] - 1s 93ms/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.4239417989417989
f1_score :  0.32352941176470584
accuracy_score :  0.3235294117647059
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [8]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 0s 453ms/step
Epoch 0, Loss: -1.5934035778045654, Accuracy: 0.0
1/1 [==============================] - 0s 238ms/step
Epoch 1, Loss: -3.858031988143921, Accuracy: 0.0
1/1 [==============================] - 0s 236ms/step
Epoch 2, Loss: -5.089278697967529, Accuracy: 0.0
1/1 [==============================] - 0s 239ms/step
Epoch 3, Loss: -7.632634162902832, Accuracy: 0.0
1/1 [==============================] - 0s 237ms/step
Epoch 4, Loss: -3.738422393798828, Accuracy: 0.0
1/1 [==============================] - 0s 232ms/step
Epoch 5, Loss: -6.138092994689941, Accuracy: 0.0
1/1 [==============================] - 0s 236ms/step
Epoch 6, Loss: -8.648360252380371, Accuracy: 0.0
1/1 [==============================] - 0s 236ms/step
Epoch 7, Loss: -13.078590393066406, Accuracy: 0.0
1/1 [==============================] - 0s 244ms/step
Epoch 8, Loss: -18.4149169921875, Accuracy: 0.0
1/1 [==============================] - 0s 231ms/step
Epoch 9, Loss: -23.876670837

1/1 [==============================] - 0s 240ms/step
Epoch 31, Loss: -4.168414115905762, Accuracy: 0.0
1/1 [==============================] - 0s 237ms/step
Epoch 32, Loss: -6.792453289031982, Accuracy: 0.0
1/1 [==============================] - 0s 232ms/step
Epoch 33, Loss: -8.756124496459961, Accuracy: 0.0
1/1 [==============================] - 0s 243ms/step
Epoch 34, Loss: -19.889545440673828, Accuracy: 0.0
1/1 [==============================] - 0s 230ms/step
Epoch 35, Loss: -24.30558967590332, Accuracy: 0.0
1/1 [==============================] - 0s 236ms/step
Epoch 36, Loss: 0.02123284339904785, Accuracy: 0.0
1/1 [==============================] - 0s 240ms/step
Epoch 37, Loss: 1.8964518308639526, Accuracy: 0.0
1/1 [==============================] - 0s 239ms/step
Epoch 38, Loss: -12.955101013183594, Accuracy: 0.0
1/1 [==============================] - 0s 233ms/step
Epoch 39, Loss: -14.149049758911133, Accuracy: 0.0
1/1 [==============================] - 0s 230ms/step
Epoch 40, Loss: 

1/1 [==============================] - 0s 240ms/step
Epoch 62, Loss: -42.935302734375, Accuracy: 0.0
1/1 [==============================] - 0s 252ms/step
Epoch 63, Loss: -43.88916778564453, Accuracy: 0.0
1/1 [==============================] - 0s 234ms/step
Epoch 64, Loss: -63.11146545410156, Accuracy: 0.0
1/1 [==============================] - 0s 234ms/step
Epoch 65, Loss: -57.60856628417969, Accuracy: 0.0
1/1 [==============================] - 0s 231ms/step
Epoch 66, Loss: -80.39241790771484, Accuracy: 0.0
1/1 [==============================] - 0s 243ms/step
Epoch 67, Loss: -93.59530639648438, Accuracy: 0.0
1/1 [==============================] - 0s 231ms/step
Epoch 68, Loss: -100.37834167480469, Accuracy: 0.0
1/1 [==============================] - 0s 233ms/step
Epoch 69, Loss: -104.44894409179688, Accuracy: 0.0
1/1 [==============================] - 0s 238ms/step
Epoch 70, Loss: -71.24906921386719, Accuracy: 0.0
1/1 [==============================] - 0s 236ms/step
Epoch 71, Loss: -87.

1/1 [==============================] - 0s 232ms/step
Epoch 93, Loss: -306.8102722167969, Accuracy: 0.0
1/1 [==============================] - 0s 236ms/step
Epoch 94, Loss: -181.22373962402344, Accuracy: 0.0
1/1 [==============================] - 0s 236ms/step
Epoch 95, Loss: -266.22906494140625, Accuracy: 0.0
1/1 [==============================] - 0s 232ms/step
Epoch 96, Loss: -297.8056640625, Accuracy: 0.0
1/1 [==============================] - 0s 233ms/step
Epoch 97, Loss: -383.7279052734375, Accuracy: 0.0
1/1 [==============================] - 0s 243ms/step
Epoch 98, Loss: -308.4736328125, Accuracy: 0.0
1/1 [==============================] - 0s 231ms/step
Epoch 99, Loss: -357.8262023925781, Accuracy: 0.0
2/2 [==============================] - 1s 49ms/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.5639097744360902
f1_score :  0.425531914893617
accuracy_score :  0.3076923076923077
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [9]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 0s 429ms/step
Epoch 0, Loss: -5.534163475036621, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 1, Loss: -13.248478889465332, Accuracy: 0.0
1/1 [==============================] - 0s 214ms/step
Epoch 2, Loss: -6.705953121185303, Accuracy: 0.0
1/1 [==============================] - 0s 237ms/step
Epoch 3, Loss: -18.511327743530273, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 4, Loss: -19.64701271057129, Accuracy: 0.0
1/1 [==============================] - 0s 234ms/step
Epoch 5, Loss: -42.961116790771484, Accuracy: 0.0
1/1 [==============================] - 0s 210ms/step
Epoch 6, Loss: -33.66911697387695, Accuracy: 0.0
1/1 [==============================] - 0s 216ms/step
Epoch 7, Loss: -67.45957946777344, Accuracy: 0.0
1/1 [==============================] - 0s 210ms/step
Epoch 8, Loss: -83.51197052001953, Accuracy: 0.0
1/1 [==============================] - 0s 206ms/step
Epoch 9, Loss: -108.083419

1/1 [==============================] - 0s 225ms/step
Epoch 22, Loss: -90.41085052490234, Accuracy: 0.0
1/1 [==============================] - 0s 214ms/step
Epoch 23, Loss: -159.175048828125, Accuracy: 0.0
1/1 [==============================] - 0s 215ms/step
Epoch 24, Loss: -154.2310791015625, Accuracy: 0.0
1/1 [==============================] - 0s 212ms/step
Epoch 25, Loss: -122.70353698730469, Accuracy: 0.0
1/1 [==============================] - 0s 209ms/step
Epoch 26, Loss: -156.7357177734375, Accuracy: 0.0
1/1 [==============================] - 0s 214ms/step
Epoch 27, Loss: -76.52642059326172, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 28, Loss: -162.1207733154297, Accuracy: 0.0
1/1 [==============================] - 0s 208ms/step
Epoch 29, Loss: -150.93994140625, Accuracy: 0.0
1/1 [==============================] - 0s 208ms/step
Epoch 30, Loss: -178.7367706298828, Accuracy: 0.0
1/1 [==============================] - 0s 212ms/step
Epoch 31, Loss: -189.0

1/1 [==============================] - 0s 212ms/step
Epoch 44, Loss: -85.3952865600586, Accuracy: 0.0
1/1 [==============================] - 0s 214ms/step
Epoch 45, Loss: -120.67601776123047, Accuracy: 0.0
1/1 [==============================] - 0s 210ms/step
Epoch 46, Loss: -205.18759155273438, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 47, Loss: -181.64112854003906, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 48, Loss: -330.71368408203125, Accuracy: 0.0
1/1 [==============================] - 0s 228ms/step
Epoch 49, Loss: -343.3091735839844, Accuracy: 0.0
1/1 [==============================] - 0s 209ms/step
Epoch 50, Loss: -351.6389465332031, Accuracy: 0.0
1/1 [==============================] - 0s 211ms/step
Epoch 51, Loss: -372.02349853515625, Accuracy: 0.0
1/1 [==============================] - 0s 209ms/step
Epoch 52, Loss: -373.4046325683594, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 53, Loss: 

1/1 [==============================] - 0s 207ms/step
Epoch 66, Loss: -749.86865234375, Accuracy: 0.0
1/1 [==============================] - 0s 252ms/step
Epoch 67, Loss: -597.7385864257812, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 68, Loss: -590.7919311523438, Accuracy: 0.0
1/1 [==============================] - 0s 221ms/step
Epoch 69, Loss: -119.30314636230469, Accuracy: 0.0
1/1 [==============================] - 0s 210ms/step
Epoch 70, Loss: -255.59657287597656, Accuracy: 0.0
1/1 [==============================] - 0s 212ms/step
Epoch 71, Loss: -354.59063720703125, Accuracy: 0.0
1/1 [==============================] - 0s 205ms/step
Epoch 72, Loss: -515.6327514648438, Accuracy: 0.0
1/1 [==============================] - 0s 210ms/step
Epoch 73, Loss: -865.5634765625, Accuracy: 0.0
1/1 [==============================] - 0s 211ms/step
Epoch 74, Loss: -1080.579345703125, Accuracy: 0.0
1/1 [==============================] - 0s 216ms/step
Epoch 75, Loss: -316.6

1/1 [==============================] - 0s 231ms/step
Epoch 88, Loss: -1536.086669921875, Accuracy: 0.0
1/1 [==============================] - 0s 208ms/step
Epoch 89, Loss: -1026.583740234375, Accuracy: 0.0
1/1 [==============================] - 0s 206ms/step
Epoch 90, Loss: -1213.4971923828125, Accuracy: 0.0
1/1 [==============================] - 0s 216ms/step
Epoch 91, Loss: -1271.698974609375, Accuracy: 0.0
1/1 [==============================] - 0s 229ms/step
Epoch 92, Loss: -942.9940185546875, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 93, Loss: -1286.5904541015625, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 94, Loss: -1048.7344970703125, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 95, Loss: -1004.4794921875, Accuracy: 0.0
1/1 [==============================] - 0s 207ms/step
Epoch 96, Loss: -1088.7711181640625, Accuracy: 0.0
1/1 [==============================] - 0s 213ms/step
Epoch 97, Loss: -4

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.7003571428571428
f1_score :  0.6190476190476191
accuracy_score :  0.5636363636363636
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [10]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 777ms/step
Epoch 0, Loss: -1.171656608581543, Accuracy: 0.0
1/1 [==============================] - 1s 576ms/step
Epoch 1, Loss: -3.898059368133545, Accuracy: 0.0
1/1 [==============================] - 1s 571ms/step
Epoch 2, Loss: -4.572388648986816, Accuracy: 0.0
1/1 [==============================] - 1s 571ms/step
Epoch 3, Loss: -6.141297817230225, Accuracy: 0.0
1/1 [==============================] - 1s 564ms/step
Epoch 4, Loss: -9.809771537780762, Accuracy: 0.0
1/1 [==============================] - 1s 572ms/step
Epoch 5, Loss: -8.301695823669434, Accuracy: 0.0
1/1 [==============================] - 1s 574ms/step
Epoch 6, Loss: -14.462043762207031, Accuracy: 0.0
1/1 [==============================] - 1s 582ms/step
Epoch 7, Loss: -22.618213653564453, Accuracy: 0.0
1/1 [==============================] - 1s 568ms/step
Epoch 8, Loss: 58.91735076904297, Accuracy: 0.0
1/1 [==============================] - 1s 578ms/step
Epoch 9, Loss: -8.1874933242

1/1 [==============================] - 1s 564ms/step
Epoch 34, Loss: -130.70208740234375, Accuracy: 0.0
1/1 [==============================] - 1s 573ms/step
Epoch 35, Loss: -110.33241271972656, Accuracy: 0.0
1/1 [==============================] - 1s 587ms/step
Epoch 36, Loss: -122.83613586425781, Accuracy: 0.0
1/1 [==============================] - 1s 565ms/step
Epoch 37, Loss: -121.64514923095703, Accuracy: 0.0
1/1 [==============================] - 1s 561ms/step
Epoch 38, Loss: -82.11229705810547, Accuracy: 0.0
1/1 [==============================] - 1s 575ms/step
Epoch 39, Loss: -117.92304992675781, Accuracy: 0.0
1/1 [==============================] - 1s 562ms/step
Epoch 40, Loss: -148.95925903320312, Accuracy: 0.0
1/1 [==============================] - 1s 563ms/step
Epoch 41, Loss: -176.12559509277344, Accuracy: 0.0
1/1 [==============================] - 1s 554ms/step
Epoch 42, Loss: -148.14285278320312, Accuracy: 0.0
1/1 [==============================] - 1s 562ms/step
Epoch 43, Lo

1/1 [==============================] - 1s 557ms/step
Epoch 69, Loss: -263.8302001953125, Accuracy: 0.0
1/1 [==============================] - 1s 563ms/step
Epoch 70, Loss: -262.0384521484375, Accuracy: 0.0
1/1 [==============================] - 1s 565ms/step
Epoch 71, Loss: -308.68524169921875, Accuracy: 0.0
1/1 [==============================] - 1s 561ms/step
Epoch 72, Loss: -247.0115966796875, Accuracy: 0.0
1/1 [==============================] - 1s 575ms/step
Epoch 73, Loss: -291.3575744628906, Accuracy: 0.0
1/1 [==============================] - 1s 594ms/step
Epoch 74, Loss: -313.2783203125, Accuracy: 0.0
1/1 [==============================] - 1s 558ms/step
Epoch 75, Loss: -303.8952331542969, Accuracy: 0.0
1/1 [==============================] - 1s 562ms/step
Epoch 76, Loss: -346.709716796875, Accuracy: 0.0
1/1 [==============================] - 1s 565ms/step
Epoch 77, Loss: -337.02484130859375, Accuracy: 0.0
1/1 [==============================] - 1s 562ms/step
Epoch 78, Loss: -368.5

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.43104619565217395
f1_score :  0.0
accuracy_score :  0.6290322580645161
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [11]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 952ms/step
Epoch 0, Loss: -1.36625075340271, Accuracy: 0.0
1/1 [==============================] - 1s 755ms/step
Epoch 1, Loss: -4.210896015167236, Accuracy: 0.0
1/1 [==============================] - 1s 746ms/step
Epoch 2, Loss: -2.9845776557922363, Accuracy: 0.0
1/1 [==============================] - 1s 756ms/step
Epoch 3, Loss: -9.52033519744873, Accuracy: 0.0
1/1 [==============================] - 1s 746ms/step
Epoch 4, Loss: -11.805108070373535, Accuracy: 0.0
1/1 [==============================] - 1s 754ms/step
Epoch 5, Loss: -15.606002807617188, Accuracy: 0.0
1/1 [==============================] - 1s 751ms/step
Epoch 6, Loss: -22.101669311523438, Accuracy: 0.0
1/1 [==============================] - 1s 751ms/step
Epoch 7, Loss: -21.29658317565918, Accuracy: 0.0
1/1 [==============================] - 1s 751ms/step
Epoch 8, Loss: -28.15079689025879, Accuracy: 0.0
1/1 [==============================] - 1s 751ms/step
Epoch 9, Loss: -38.73691177

1/1 [==============================] - 1s 748ms/step
Epoch 38, Loss: -397.2948303222656, Accuracy: 0.0
1/1 [==============================] - 1s 819ms/step
Epoch 39, Loss: -327.9558410644531, Accuracy: 0.0
1/1 [==============================] - 1s 754ms/step
Epoch 40, Loss: -376.39837646484375, Accuracy: 0.0
1/1 [==============================] - 1s 743ms/step
Epoch 41, Loss: -424.4626159667969, Accuracy: 0.0
1/1 [==============================] - 1s 744ms/step
Epoch 42, Loss: -439.4646301269531, Accuracy: 0.0
1/1 [==============================] - 1s 752ms/step
Epoch 43, Loss: -425.3738708496094, Accuracy: 0.0
1/1 [==============================] - 1s 748ms/step
Epoch 44, Loss: -475.92230224609375, Accuracy: 0.0
1/1 [==============================] - 1s 757ms/step
Epoch 45, Loss: -528.68359375, Accuracy: 0.0
1/1 [==============================] - 1s 743ms/step
Epoch 46, Loss: -568.1478881835938, Accuracy: 0.0
1/1 [==============================] - 1s 751ms/step
Epoch 47, Loss: -648.51

1/1 [==============================] - 1s 746ms/step
Epoch 58, Loss: -902.1781005859375, Accuracy: 0.0
1/1 [==============================] - 1s 747ms/step
Epoch 59, Loss: -736.1840209960938, Accuracy: 0.0
1/1 [==============================] - 1s 789ms/step
Epoch 60, Loss: -853.037109375, Accuracy: 0.0
1/1 [==============================] - 1s 758ms/step
Epoch 61, Loss: -925.4887084960938, Accuracy: 0.0
1/1 [==============================] - 1s 751ms/step
Epoch 62, Loss: -917.7606201171875, Accuracy: 0.0
1/1 [==============================] - 1s 748ms/step
Epoch 63, Loss: -1113.6724853515625, Accuracy: 0.0
1/1 [==============================] - 1s 745ms/step
Epoch 64, Loss: -1149.2264404296875, Accuracy: 0.0
1/1 [==============================] - 1s 752ms/step
Epoch 65, Loss: -824.8452758789062, Accuracy: 0.0
1/1 [==============================] - 1s 762ms/step
Epoch 66, Loss: -1083.570068359375, Accuracy: 0.0
1/1 [==============================] - 1s 749ms/step
Epoch 67, Loss: -1230.

1/1 [==============================] - 1s 745ms/step
Epoch 97, Loss: -1630.928466796875, Accuracy: 0.0
1/1 [==============================] - 1s 752ms/step
Epoch 98, Loss: 1838.3265380859375, Accuracy: 0.0
1/1 [==============================] - 1s 743ms/step
Epoch 99, Loss: -1529.0418701171875, Accuracy: 0.0
1/1 [==============================] - 1s 726ms/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.40273704789833825
f1_score :  0.2608695652173913
accuracy_score :  0.2608695652173913
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [14]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

# X_train, Y_train, G = train_gan_data(X_train)
# x_train = np.reshape(X_train, (len(X_train), -1))
# x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.7411347517730497
f1_score :  0.3779527559055118
accuracy_score :  0.5269461077844312
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [15]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 854ms/step
Epoch 0, Loss: -0.9832137823104858, Accuracy: 0.0
1/1 [==============================] - 1s 903ms/step
Epoch 1, Loss: -2.4129955768585205, Accuracy: 0.0
1/1 [==============================] - 1s 849ms/step
Epoch 2, Loss: -5.765809059143066, Accuracy: 0.0
1/1 [==============================] - 1s 853ms/step
Epoch 3, Loss: -10.928083419799805, Accuracy: 0.0
1/1 [==============================] - 1s 851ms/step
Epoch 4, Loss: -10.245390892028809, Accuracy: 0.0
1/1 [==============================] - 1s 849ms/step
Epoch 5, Loss: 0.37631070613861084, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 6, Loss: -1.162635326385498, Accuracy: 0.0
1/1 [==============================] - 1s 844ms/step
Epoch 7, Loss: -2.837214946746826, Accuracy: 0.0
1/1 [==============================] - 1s 851ms/step
Epoch 8, Loss: -5.1397857666015625, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 9, Loss: -5.80688

1/1 [==============================] - 1s 868ms/step
Epoch 28, Loss: 112.07859802246094, Accuracy: 0.0
1/1 [==============================] - 1s 863ms/step
Epoch 29, Loss: 430.49700927734375, Accuracy: 0.0
1/1 [==============================] - 1s 844ms/step
Epoch 30, Loss: -42.688812255859375, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 31, Loss: -63.12225341796875, Accuracy: 0.0
1/1 [==============================] - 1s 840ms/step
Epoch 32, Loss: -74.60047912597656, Accuracy: 0.0
1/1 [==============================] - 1s 845ms/step
Epoch 33, Loss: -50.747718811035156, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 34, Loss: -78.07185363769531, Accuracy: 0.0
1/1 [==============================] - 1s 909ms/step
Epoch 35, Loss: -80.88983154296875, Accuracy: 0.0
1/1 [==============================] - 1s 843ms/step
Epoch 36, Loss: -81.076416015625, Accuracy: 0.0
1/1 [==============================] - 1s 867ms/step
Epoch 37, Loss: -84.

1/1 [==============================] - 1s 850ms/step
Epoch 56, Loss: -199.82090759277344, Accuracy: 0.0
1/1 [==============================] - 1s 844ms/step
Epoch 57, Loss: -265.6846618652344, Accuracy: 0.0
1/1 [==============================] - 1s 858ms/step
Epoch 58, Loss: -266.68572998046875, Accuracy: 0.0
1/1 [==============================] - 1s 848ms/step
Epoch 59, Loss: -311.12884521484375, Accuracy: 0.0
1/1 [==============================] - 1s 856ms/step
Epoch 60, Loss: -240.54470825195312, Accuracy: 0.0
1/1 [==============================] - 1s 855ms/step
Epoch 61, Loss: -295.817138671875, Accuracy: 0.0
1/1 [==============================] - 1s 850ms/step
Epoch 62, Loss: -329.5383605957031, Accuracy: 0.0
1/1 [==============================] - 1s 843ms/step
Epoch 63, Loss: -380.6512145996094, Accuracy: 0.0
1/1 [==============================] - 1s 854ms/step
Epoch 64, Loss: -346.07489013671875, Accuracy: 0.0
1/1 [==============================] - 1s 865ms/step
Epoch 65, Loss: 

1/1 [==============================] - 1s 1s/step
Epoch 84, Loss: -685.790283203125, Accuracy: 0.0
1/1 [==============================] - 1s 847ms/step
Epoch 85, Loss: -596.8363647460938, Accuracy: 0.0
1/1 [==============================] - 1s 847ms/step
Epoch 86, Loss: -787.7403564453125, Accuracy: 0.0
1/1 [==============================] - 1s 857ms/step
Epoch 87, Loss: 246.62564086914062, Accuracy: 0.0
1/1 [==============================] - 1s 856ms/step
Epoch 88, Loss: -601.5697021484375, Accuracy: 0.0
1/1 [==============================] - 1s 850ms/step
Epoch 89, Loss: -726.0064697265625, Accuracy: 0.0
1/1 [==============================] - 1s 840ms/step
Epoch 90, Loss: -776.5250854492188, Accuracy: 0.0
1/1 [==============================] - 1s 857ms/step
Epoch 91, Loss: -740.4808349609375, Accuracy: 0.0
1/1 [==============================] - 1s 842ms/step
Epoch 92, Loss: -720.7542724609375, Accuracy: 0.0
1/1 [==============================] - 1s 849ms/step
Epoch 93, Loss: -764.951

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.056773929083828646
f1_score :  0.12578616352201258
accuracy_score :  0.13125
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [16]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 0s 414ms/step
Epoch 0, Loss: -1.5125255584716797, Accuracy: 0.0
1/1 [==============================] - 0s 180ms/step
Epoch 1, Loss: -3.056988477706909, Accuracy: 0.0
1/1 [==============================] - 0s 182ms/step
Epoch 2, Loss: -7.8785576820373535, Accuracy: 0.0
1/1 [==============================] - 0s 181ms/step
Epoch 3, Loss: -6.626068115234375, Accuracy: 0.0
1/1 [==============================] - 0s 186ms/step
Epoch 4, Loss: -13.061779975891113, Accuracy: 0.0
1/1 [==============================] - 0s 191ms/step
Epoch 5, Loss: -16.71512222290039, Accuracy: 0.0
1/1 [==============================] - 0s 185ms/step
Epoch 6, Loss: 12.289020538330078, Accuracy: 0.0
1/1 [==============================] - 0s 190ms/step
Epoch 7, Loss: -2.3271026611328125, Accuracy: 0.0
1/1 [==============================] - 0s 194ms/step
Epoch 8, Loss: -6.082914352416992, Accuracy: 0.0
1/1 [==============================] - 0s 184ms/step
Epoch 9, Loss: -11.215161

1/1 [==============================] - 0s 184ms/step
Epoch 34, Loss: -60.04136657714844, Accuracy: 0.0
1/1 [==============================] - 0s 190ms/step
Epoch 35, Loss: -78.10762786865234, Accuracy: 0.0
1/1 [==============================] - 0s 191ms/step
Epoch 36, Loss: -37.409576416015625, Accuracy: 0.0
1/1 [==============================] - 0s 186ms/step
Epoch 37, Loss: 205.07240295410156, Accuracy: 0.0
1/1 [==============================] - 0s 187ms/step
Epoch 38, Loss: -7.8958964347839355, Accuracy: 0.0
1/1 [==============================] - 0s 178ms/step
Epoch 39, Loss: -38.38429260253906, Accuracy: 0.0
1/1 [==============================] - 0s 191ms/step
Epoch 40, Loss: -33.82194519042969, Accuracy: 0.0
1/1 [==============================] - 0s 179ms/step
Epoch 41, Loss: -34.941017150878906, Accuracy: 0.0
1/1 [==============================] - 0s 193ms/step
Epoch 42, Loss: -34.44184494018555, Accuracy: 0.0
1/1 [==============================] - 0s 186ms/step
Epoch 43, Loss: -

1/1 [==============================] - 0s 181ms/step
Epoch 69, Loss: -137.5788116455078, Accuracy: 0.0
1/1 [==============================] - 0s 188ms/step
Epoch 70, Loss: -73.13715362548828, Accuracy: 0.0
1/1 [==============================] - 0s 185ms/step
Epoch 71, Loss: -130.24639892578125, Accuracy: 0.0
1/1 [==============================] - 0s 185ms/step
Epoch 72, Loss: -99.22412109375, Accuracy: 0.0
1/1 [==============================] - 0s 183ms/step
Epoch 73, Loss: -174.66773986816406, Accuracy: 0.0
1/1 [==============================] - 0s 186ms/step
Epoch 74, Loss: -154.97705078125, Accuracy: 0.0
1/1 [==============================] - 0s 179ms/step
Epoch 75, Loss: -149.77919006347656, Accuracy: 0.0
1/1 [==============================] - 0s 188ms/step
Epoch 76, Loss: -104.74671173095703, Accuracy: 0.0
1/1 [==============================] - 0s 181ms/step
Epoch 77, Loss: -147.09559631347656, Accuracy: 0.0
1/1 [==============================] - 0s 203ms/step
Epoch 78, Loss: -160

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.7297979797979798
f1_score :  0.5
accuracy_score :  0.4700854700854701
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [17]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 948ms/step
Epoch 0, Loss: -0.8560744524002075, Accuracy: 0.0
1/1 [==============================] - 1s 748ms/step
Epoch 1, Loss: -0.9074727296829224, Accuracy: 0.0
1/1 [==============================] - 1s 741ms/step
Epoch 2, Loss: -1.327478289604187, Accuracy: 0.0
1/1 [==============================] - 1s 740ms/step
Epoch 3, Loss: -2.3378398418426514, Accuracy: 0.0
1/1 [==============================] - 1s 768ms/step
Epoch 4, Loss: -2.904956340789795, Accuracy: 0.0
1/1 [==============================] - 1s 748ms/step
Epoch 5, Loss: -3.9116947650909424, Accuracy: 0.0
1/1 [==============================] - 1s 737ms/step
Epoch 6, Loss: -2.4872212409973145, Accuracy: 0.0
1/1 [==============================] - 1s 762ms/step
Epoch 7, Loss: -2.999105215072632, Accuracy: 0.0
1/1 [==============================] - 1s 736ms/step
Epoch 8, Loss: -4.115680694580078, Accuracy: 0.0
1/1 [==============================] - 1s 746ms/step
Epoch 9, Loss: -5.993591

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.8722222222222222
f1_score :  0.6896551724137931
accuracy_score :  0.7857142857142857
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [18]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 760ms/step
Epoch 0, Loss: -1.696805715560913, Accuracy: 0.0
1/1 [==============================] - 1s 565ms/step
Epoch 1, Loss: -3.021306037902832, Accuracy: 0.0
1/1 [==============================] - 1s 564ms/step
Epoch 2, Loss: -5.347493648529053, Accuracy: 0.0
1/1 [==============================] - 1s 565ms/step
Epoch 3, Loss: -7.979937553405762, Accuracy: 0.0
1/1 [==============================] - 1s 570ms/step
Epoch 4, Loss: -14.027647018432617, Accuracy: 0.0
1/1 [==============================] - 1s 577ms/step
Epoch 5, Loss: -15.907262802124023, Accuracy: 0.0
1/1 [==============================] - 1s 570ms/step
Epoch 6, Loss: -9.80473804473877, Accuracy: 0.0
1/1 [==============================] - 1s 579ms/step
Epoch 7, Loss: -15.263405799865723, Accuracy: 0.0
1/1 [==============================] - 1s 560ms/step
Epoch 8, Loss: -28.28793716430664, Accuracy: 0.0
1/1 [==============================] - 1s 567ms/step
Epoch 9, Loss: -30.39939117

1/1 [==============================] - 1s 566ms/step
Epoch 38, Loss: -46.27031707763672, Accuracy: 0.0
1/1 [==============================] - 1s 572ms/step
Epoch 39, Loss: 14.319002151489258, Accuracy: 0.0
1/1 [==============================] - 1s 578ms/step
Epoch 40, Loss: -22.07862663269043, Accuracy: 0.0
1/1 [==============================] - 1s 569ms/step
Epoch 41, Loss: -47.09526062011719, Accuracy: 0.0
1/1 [==============================] - 1s 568ms/step
Epoch 42, Loss: -59.56026077270508, Accuracy: 0.0
1/1 [==============================] - 1s 566ms/step
Epoch 43, Loss: -69.62002563476562, Accuracy: 0.0
1/1 [==============================] - 1s 567ms/step
Epoch 44, Loss: -84.39988708496094, Accuracy: 0.0
1/1 [==============================] - 1s 571ms/step
Epoch 45, Loss: -80.30061340332031, Accuracy: 0.0
1/1 [==============================] - 1s 566ms/step
Epoch 46, Loss: -110.99553680419922, Accuracy: 0.0
1/1 [==============================] - 1s 565ms/step
Epoch 47, Loss: -98

1/1 [==============================] - 1s 567ms/step
Epoch 77, Loss: -278.05609130859375, Accuracy: 0.0
1/1 [==============================] - 1s 564ms/step
Epoch 78, Loss: -312.0436706542969, Accuracy: 0.0
1/1 [==============================] - 1s 559ms/step
Epoch 79, Loss: -327.3646545410156, Accuracy: 0.0
1/1 [==============================] - 1s 564ms/step
Epoch 80, Loss: -314.9847412109375, Accuracy: 0.0
1/1 [==============================] - 1s 558ms/step
Epoch 81, Loss: -407.32904052734375, Accuracy: 0.0
1/1 [==============================] - 1s 573ms/step
Epoch 82, Loss: -381.33929443359375, Accuracy: 0.0
1/1 [==============================] - 1s 563ms/step
Epoch 83, Loss: -380.3401184082031, Accuracy: 0.0
1/1 [==============================] - 1s 572ms/step
Epoch 84, Loss: -369.1914367675781, Accuracy: 0.0
1/1 [==============================] - 1s 572ms/step
Epoch 85, Loss: -466.75592041015625, Accuracy: 0.0
1/1 [==============================] - 1s 560ms/step
Epoch 86, Loss: 

1/1 [==============================] - 1s 558ms/step
Epoch 97, Loss: -407.9181213378906, Accuracy: 0.0
1/1 [==============================] - 1s 572ms/step
Epoch 98, Loss: -347.1761169433594, Accuracy: 0.0
1/1 [==============================] - 1s 578ms/step
Epoch 99, Loss: -507.4490661621094, Accuracy: 0.0
1/1 [==============================] - 1s 693ms/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.7079166666666666
f1_score :  0.7786259541984734
accuracy_score :  0.71
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [19]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 822ms/step
Epoch 0, Loss: -1.0563476085662842, Accuracy: 0.0
1/1 [==============================] - 1s 624ms/step
Epoch 1, Loss: -4.4037675857543945, Accuracy: 0.0
1/1 [==============================] - 1s 615ms/step
Epoch 2, Loss: -6.719503879547119, Accuracy: 0.0
1/1 [==============================] - 1s 617ms/step
Epoch 3, Loss: -7.063434600830078, Accuracy: 0.0
1/1 [==============================] - 1s 613ms/step
Epoch 4, Loss: -5.930355548858643, Accuracy: 0.0
1/1 [==============================] - 1s 633ms/step
Epoch 5, Loss: -9.664938926696777, Accuracy: 0.0
1/1 [==============================] - 1s 618ms/step
Epoch 6, Loss: -10.125481605529785, Accuracy: 0.0
1/1 [==============================] - 1s 617ms/step
Epoch 7, Loss: -14.09066104888916, Accuracy: 0.0
1/1 [==============================] - 1s 619ms/step
Epoch 8, Loss: -14.896671295166016, Accuracy: 0.0
1/1 [==============================] - 1s 615ms/step
Epoch 9, Loss: -18.369104

1/1 [==============================] - 1s 634ms/step
Epoch 34, Loss: -73.4056625366211, Accuracy: 0.0
1/1 [==============================] - 1s 614ms/step
Epoch 35, Loss: -78.29617309570312, Accuracy: 0.0
1/1 [==============================] - 1s 623ms/step
Epoch 36, Loss: -109.1553726196289, Accuracy: 0.0
1/1 [==============================] - 1s 631ms/step
Epoch 37, Loss: -95.99561309814453, Accuracy: 0.0
1/1 [==============================] - 1s 668ms/step
Epoch 38, Loss: -78.88417053222656, Accuracy: 0.0
1/1 [==============================] - 1s 625ms/step
Epoch 39, Loss: -126.16954040527344, Accuracy: 0.0
1/1 [==============================] - 1s 604ms/step
Epoch 40, Loss: -121.72637176513672, Accuracy: 0.0
1/1 [==============================] - 1s 621ms/step
Epoch 41, Loss: -135.40480041503906, Accuracy: 0.0
1/1 [==============================] - 1s 622ms/step
Epoch 42, Loss: -136.59759521484375, Accuracy: 0.0
1/1 [==============================] - 1s 617ms/step
Epoch 43, Loss: -

1/1 [==============================] - 1s 633ms/step
Epoch 69, Loss: -477.8077392578125, Accuracy: 0.0
1/1 [==============================] - 1s 651ms/step
Epoch 70, Loss: -395.3691101074219, Accuracy: 0.0
1/1 [==============================] - 1s 617ms/step
Epoch 71, Loss: -400.532958984375, Accuracy: 0.0
1/1 [==============================] - 1s 626ms/step
Epoch 72, Loss: -372.2969970703125, Accuracy: 0.0
1/1 [==============================] - 1s 641ms/step
Epoch 73, Loss: -539.5521850585938, Accuracy: 0.0
1/1 [==============================] - 1s 615ms/step
Epoch 74, Loss: -551.9735107421875, Accuracy: 0.0
1/1 [==============================] - 1s 610ms/step
Epoch 75, Loss: -455.4101257324219, Accuracy: 0.0
1/1 [==============================] - 1s 621ms/step
Epoch 76, Loss: -403.698486328125, Accuracy: 0.0
1/1 [==============================] - 1s 616ms/step
Epoch 77, Loss: -586.5125732421875, Accuracy: 0.0
1/1 [==============================] - 1s 614ms/step
Epoch 78, Loss: -570.4

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.9280701754385965
f1_score :  0.5428571428571428
accuracy_score :  0.5949367088607594
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [20]:
# X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

X_train, Y_train, G = train_gan_data(X_train)
x_train = np.reshape(X_train, (len(X_train), -1))
x_test = np.reshape(X_test, (len(X_test), -1))
result = OCSVM(x_train, x_test, Y_train, y_ts)
res.append(result)

1/1 [==============================] - 1s 644ms/step
Epoch 0, Loss: -1.8424484729766846, Accuracy: 0.0
1/1 [==============================] - 0s 452ms/step
Epoch 1, Loss: -4.916791915893555, Accuracy: 0.0
1/1 [==============================] - 0s 437ms/step
Epoch 2, Loss: -8.811084747314453, Accuracy: 0.0
1/1 [==============================] - 0s 456ms/step
Epoch 3, Loss: 1.55038321018219, Accuracy: 0.0
1/1 [==============================] - 0s 442ms/step
Epoch 4, Loss: -12.35798454284668, Accuracy: 0.0
1/1 [==============================] - 0s 466ms/step
Epoch 5, Loss: -3.9628775119781494, Accuracy: 0.0
1/1 [==============================] - 0s 452ms/step
Epoch 6, Loss: -13.819273948669434, Accuracy: 0.0
1/1 [==============================] - 0s 443ms/step
Epoch 7, Loss: -25.61874008178711, Accuracy: 0.0
1/1 [==============================] - 0s 438ms/step
Epoch 8, Loss: -22.416858673095703, Accuracy: 0.0
1/1 [==============================] - 0s 474ms/step
Epoch 9, Loss: -39.39002990

1/1 [==============================] - 0s 446ms/step
Epoch 34, Loss: -144.20401000976562, Accuracy: 0.0
1/1 [==============================] - 0s 445ms/step
Epoch 35, Loss: -116.76018524169922, Accuracy: 0.0
1/1 [==============================] - 0s 465ms/step
Epoch 36, Loss: -241.7569580078125, Accuracy: 0.0
1/1 [==============================] - 0s 444ms/step
Epoch 37, Loss: -251.50323486328125, Accuracy: 0.0
1/1 [==============================] - 0s 444ms/step
Epoch 38, Loss: -310.5233459472656, Accuracy: 0.0
1/1 [==============================] - 0s 442ms/step
Epoch 39, Loss: -338.75433349609375, Accuracy: 0.0
1/1 [==============================] - 0s 437ms/step
Epoch 40, Loss: -288.13006591796875, Accuracy: 0.0
1/1 [==============================] - 0s 441ms/step
Epoch 41, Loss: -338.52130126953125, Accuracy: 0.0
1/1 [==============================] - 0s 442ms/step
Epoch 42, Loss: -277.69696044921875, Accuracy: 0.0
1/1 [==============================] - 0s 480ms/step
Epoch 43, Los

1/1 [==============================] - 0s 474ms/step
Epoch 69, Loss: -713.6973876953125, Accuracy: 0.0
1/1 [==============================] - 0s 434ms/step
Epoch 70, Loss: -651.1773681640625, Accuracy: 0.0
1/1 [==============================] - 0s 450ms/step
Epoch 71, Loss: -499.5390625, Accuracy: 0.0
1/1 [==============================] - 0s 432ms/step
Epoch 72, Loss: -852.8173217773438, Accuracy: 0.0
1/1 [==============================] - 0s 441ms/step
Epoch 73, Loss: -466.01904296875, Accuracy: 0.0
1/1 [==============================] - 0s 439ms/step
Epoch 74, Loss: -1041.9981689453125, Accuracy: 0.0
1/1 [==============================] - 0s 436ms/step
Epoch 75, Loss: -950.7789306640625, Accuracy: 0.0
1/1 [==============================] - 0s 439ms/step
Epoch 76, Loss: -817.2517700195312, Accuracy: 0.0
1/1 [==============================] - 0s 440ms/step
Epoch 77, Loss: -1059.736572265625, Accuracy: 0.0
1/1 [==============================] - 0s 443ms/step
Epoch 78, Loss: -877.885192

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.5677521008403361
f1_score :  0.3424657534246575
accuracy_score :  0.36423841059602646
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.25}


In [21]:
df = pd.DataFrame(res)
df.to_csv('GANOCSVMF.csv')

In [22]:
df

,0,1,2
0,0.845238,0.577778,0.771084
1,0.696594,0.654088,0.607143
2,0.530913,0.297521,0.356061
3,0.423942,0.323529,0.323529
4,0.563910,0.425532,0.307692
5,0.700357,0.619048,0.563636
6,0.431046,0.000000,0.629032
7,0.402737,0.260870,0.260870
8,0.741135,0.377953,0.526946
9,0.056774,0.125786,0.131250
